# Maryland Total Migration (FY2020-FY2024)

Dataset Source: [Maryland Open Data Portal](https://opendata.maryland.gov/api/v3/views/3hb2-c6rg/query.csv)

This notebook downloads Maryland Total Migration data by fiscal year and stores it in a SQL database.

**Dataset Overview:**
- **Time Period:** Fiscal Year 2020-2024 (FY2020 = July 1, 2019 - June 30, 2020)
- **Fiscal Year:** Maryland's fiscal year runs from July 1 to June 30
- **Frequency:** Annual (by fiscal year)
- **Geographic Coverage:** Statewide and all 24 Maryland jurisdictions (23 counties + Baltimore City)
- **Data Source:** Maryland Open Data Portal
- **Provider:** Maryland Department of Planning
- **Data Type:** Net migration (positive = net in-migration, negative = net out-migration)

In [12]:
# Import required libraries
import os
import io
import warnings
import pandas as pd
import requests
import dotenv
import mssql_python
from math import ceil

dotenv.load_dotenv()

# API Configuration
API_URL = "https://opendata.maryland.gov/api/v3/views/3hb2-c6rg/query.csv"
APP_TOKEN = os.getenv("MD_APP_TOKEN")

# Database Configuration
SQL_CONNECTION_STRING = os.getenv("SQL_CONNECTION_STRING")
if not SQL_CONNECTION_STRING:
    raise ValueError(
        "SQL_CONNECTION_STRING environment variable is required. "
        "Set it in your .env file or as an environment variable."
    )

TABLE_NAME = "[Maryland].[dbo].[Migration]"

print("✓ All imports and configuration loaded")
print("✓ Dataset: Maryland Total Migration (Fiscal Years)")
print("✓ Source: Maryland Open Data Portal, Maryland Department of Planning")
print("✓ Time Period: FY2020-FY2024 (Maryland FY runs July 1 - June 30)")
print("✓ Data Type: Net migration counts per jurisdiction")

✓ All imports and configuration loaded
✓ Dataset: Maryland Total Migration (Fiscal Years)
✓ Source: Maryland Open Data Portal, Maryland Department of Planning
✓ Time Period: FY2020-FY2024 (Maryland FY runs July 1 - June 30)
✓ Data Type: Net migration counts per jurisdiction


## Step 1: Download Data from Maryland Open Data Portal

In [16]:
# Download data from Maryland Open Data Portal - CSV format
headers = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

print("Downloading data from Maryland Open Data Portal...")
response = requests.get(API_URL, params={"accessType": "DOWNLOAD"}, headers=headers, timeout=120)
response.raise_for_status()

# Load CSV data into DataFrame
df = pd.read_csv(io.BytesIO(response.content))

print(f"✓ Dataset loaded: {len(df):,} rows × {len(df.columns)} columns\n")
print("Random sample of 5 records:")
df.sample(min(5, len(df)))

✓ Dataset loaded: 6 rows × 32 columns

Random sample of 5 records:
✓ Dataset loaded: 6 rows × 32 columns

Random sample of 5 records:


,:id,:version,:created_at,:updated_at,date_created,year,r,maryland,allegany_county,anne_arundel_county,...,kent_county,montgomery_county,prince_george_s_county,queen_anne_s_county,somerset_county,st_mary_s_county,talbot_county,washington_county,wicomico_county,worcester_county
0,row-akt7_nq5a.mqdr,rv-2sis~6hcd.5ujp,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2025-08-26T00:00:00.000,2020-04-01T00:00:00.000,2020-06-30T00:00:00.000,-4982,-58,141,...,-13,-1732,-2058,127,-28,133,36,-27,-89,135
4,row-3j6s-5wqk.tey5,rv-swvc-cz3e.irvp,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2025-08-26T00:00:00.000,2023-07-01T00:00:00.000,2024-06-30T00:00:00.000,34591,252,416,...,188,8401,7059,1003,264,630,418,1438,481,442
3,row-y5yh~42vj_9qmv,rv-6e49.w9h3.p5rc,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2025-08-26T00:00:00.000,2022-07-01T00:00:00.000,2023-06-30T00:00:00.000,11010,500,-833,...,176,3158,-776,850,406,74,319,1012,294,516
5,row-nua4.itz8.bc4k,rv-auuu~s85u-p5sa,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2025-08-26T00:00:00.000,2020-04-01T00:00:00.000,2024-06-30T00:00:00.000,33748,1006,1273,...,912,488,-14293,3720,952,1009,1656,3974,1934,3213
1,row-cpxd~54vm-v8z9,rv-59az_6vdd.dz4s,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2025-08-26T00:00:00.000,2020-07-01T00:00:00.000,2021-06-30T00:00:00.000,-8725,209,784,...,311,-8998,-11763,879,92,229,501,693,507,1264


## Step 2: Data Preparation and Cleaning

In [3]:
# Examine the DataFrame structure
print("Dataset Information:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData Types:")
print(df.dtypes)

# Exclude Date Created metadata column for display
metadata_cols_display = ['Date Created', 'date_created']
df_display = df[[col for col in df.columns if col not in metadata_cols_display]]

print(f"\nFirst few rows (excluding metadata):")
df_display.head()

Dataset Information:
Shape: (6, 32)

Columns: [':id', ':version', ':created_at', ':updated_at', 'date_created', 'year', 'r', 'maryland', 'allegany_county', 'anne_arundel_county', 'baltimore_city', 'baltimore_county', 'calvert_county', 'caroline_county', 'carroll_county', 'cecil_county', 'charles_county', 'dorchester_county', 'frederick_county', 'garrett_county', 'harford_county', 'howard_county', 'kent_county', 'montgomery_county', 'prince_george_s_county', 'queen_anne_s_county', 'somerset_county', 'st_mary_s_county', 'talbot_county', 'washington_county', 'wicomico_county', 'worcester_county']

Data Types:
:id                       object
:version                  object
:created_at               object
:updated_at               object
date_created              object
year                      object
r                         object
maryland                   int64
allegany_county            int64
anne_arundel_county        int64
baltimore_city             int64
baltimore_county       

,:id,:version,:created_at,:updated_at,year,r,maryland,allegany_county,anne_arundel_county,baltimore_city,...,kent_county,montgomery_county,prince_george_s_county,queen_anne_s_county,somerset_county,st_mary_s_county,talbot_county,washington_county,wicomico_county,worcester_county
0,row-akt7_nq5a.mqdr,rv-2sis~6hcd.5ujp,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2020-04-01T00:00:00.000,2020-06-30T00:00:00.000,-4982,-58,141,-2161,...,-13,-1732,-2058,127,-28,133,36,-27,-89,135
1,row-cpxd~54vm-v8z9,rv-59az_6vdd.dz4s,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2020-07-01T00:00:00.000,2021-06-30T00:00:00.000,-8725,209,784,-6001,...,311,-8998,-11763,879,92,229,501,693,507,1264
2,row-xwyv_whwf-nxgm,rv-xi2c-rgq5-ky7d,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2021-07-01T00:00:00.000,2022-06-30T00:00:00.000,1854,103,765,-5830,...,250,-341,-6755,861,218,-57,382,858,741,856
3,row-y5yh~42vj_9qmv,rv-6e49.w9h3.p5rc,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2022-07-01T00:00:00.000,2023-06-30T00:00:00.000,11010,500,-833,-3681,...,176,3158,-776,850,406,74,319,1012,294,516
4,row-3j6s-5wqk.tey5,rv-swvc-cz3e.irvp,2025-11-21T23:37:01.310Z,2025-11-21T23:37:01.310Z,2023-07-01T00:00:00.000,2024-06-30T00:00:00.000,34591,252,416,237,...,188,8401,7059,1003,264,630,418,1438,481,442


In [17]:
# Debug: Check year column values
print("Year column unique values:")
print(df['year'].unique())
print("\nYear column dtype:", df['year'].dtype)
print("\nFirst few rows with year:")
print(df[['year']].head(10))

Year column unique values:
['2020-04-01T00:00:00.000' '2020-07-01T00:00:00.000'
 '2021-07-01T00:00:00.000' '2022-07-01T00:00:00.000'
 '2023-07-01T00:00:00.000']

Year column dtype: object

First few rows with year:
                      year
0  2020-04-01T00:00:00.000
1  2020-07-01T00:00:00.000
2  2021-07-01T00:00:00.000
3  2022-07-01T00:00:00.000
4  2023-07-01T00:00:00.000
5  2020-04-01T00:00:00.000


In [18]:
# Check all columns to understand the date structure
print("All columns in dataset:")
print(df.columns.tolist())
print("\nFirst row of data:")
print(df.iloc[0])
print("\n" + "="*80)
print("\nChecking for date-related columns:")
for col in df.columns:
    if 'date' in col.lower() or 'year' in col.lower():
        print(f"\n{col}:")
        print(df[col].head())

All columns in dataset:
[':id', ':version', ':created_at', ':updated_at', 'date_created', 'year', 'r', 'maryland', 'allegany_county', 'anne_arundel_county', 'baltimore_city', 'baltimore_county', 'calvert_county', 'caroline_county', 'carroll_county', 'cecil_county', 'charles_county', 'dorchester_county', 'frederick_county', 'garrett_county', 'harford_county', 'howard_county', 'kent_county', 'montgomery_county', 'prince_george_s_county', 'queen_anne_s_county', 'somerset_county', 'st_mary_s_county', 'talbot_county', 'washington_county', 'wicomico_county', 'worcester_county']

First row of data:
:id                             row-akt7_nq5a.mqdr
:version                         rv-2sis~6hcd.5ujp
:created_at               2025-11-21T23:37:01.310Z
:updated_at               2025-11-21T23:37:01.310Z
date_created               2025-08-26T00:00:00.000
year                       2020-04-01T00:00:00.000
r                          2020-06-30T00:00:00.000
maryland                                    

In [26]:
# Clean column names - convert to lowercase with underscores
df.columns = [col.strip().lower().replace(" ", "_").replace("/", "_").replace("-", "_") for col in df.columns]

# Drop Socrata metadata columns (columns starting with ':')
df = df[[col for col in df.columns if not col.startswith(':')]]

# Remove 'date_created' metadata column
if 'date_created' in df.columns:
    df = df.drop(columns=['date_created'])

# Process fiscal year dates
# Maryland fiscal year runs from July 1 to June 30
# 'year' column contains start dates (e.g., 2020-07-01 = FY2021)
# 'r' column contains end dates (e.g., 2021-06-30)
if 'year' in df.columns and 'r' in df.columns:
    # Convert to datetime
    df['start_date'] = pd.to_datetime(df['year'])
    df['end_date'] = pd.to_datetime(df['r'])
    
    # Extract fiscal year from start date
    # If start date is July 1, 2020, fiscal year is 2021 (FY runs July 1, 2020 to June 30, 2021)
    # For dates starting July 1 or later, FY = year + 1
    # For dates before July 1, FY = year (special case like April-June 2020)
    df['fiscal_year'] = df['start_date'].apply(
        lambda x: x.year + 1 if x.month >= 7 else x.year
    )
    
    # Keep original date columns for reference
    df['period_start'] = df['start_date'].dt.strftime('%Y-%m-%d')
    df['period_end'] = df['end_date'].dt.strftime('%Y-%m-%d')
    
    # Drop temporary columns and original date columns
    df = df.drop(columns=['year', 'r', 'start_date', 'end_date'])
    
    # Rename fiscal_year to year for consistency with other notebooks
    df = df.rename(columns={'fiscal_year': 'year'})

# All columns except year and period columns should be numeric (migration counts)
metadata_cols = ['year', 'period_start', 'period_end']
numeric_cols = [col for col in df.columns if col not in metadata_cols]

# Convert numeric columns to integer (migration data should be whole numbers)
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# Remove rows where year is null
df = df[df['year'].notna()]

# Data quality filter: Remove rows with invalid period ranges
# Filter out row where end date is 2024-06-30 but fiscal year is 2020 (data error)
df['period_end_dt'] = pd.to_datetime(df['period_end'])
df['period_start_dt'] = pd.to_datetime(df['period_start'])
# Remove rows where period spans more than 1 year (366 days)
df['period_days'] = (df['period_end_dt'] - df['period_start_dt']).dt.days
df = df[df['period_days'] <= 366]
df = df.drop(columns=['period_end_dt', 'period_start_dt', 'period_days'])
print(f"✓ Data quality filter applied: removed rows with invalid period ranges")

# Sort by year
df = df.sort_values('year').reset_index(drop=True)

print(f"✓ Data cleaned and prepared")
print(f"✓ Final shape: {df.shape}")
if len(df) > 0:
    print(f"✓ Fiscal Year range: FY{df['year'].min()} to FY{df['year'].max()}")
    print(f"✓ Period coverage:")
    for _, row in df.iterrows():
        print(f"   FY{row['year']}: {row['period_start']} to {row['period_end']}")
else:
    print("⚠ No data remaining after cleaning")

print(f"\nCleaned column names:")
print(list(df.columns))
print(f"\nSample data:")
df.head(3)

✓ Data quality filter applied: removed rows with invalid period ranges
✓ Data cleaned and prepared
✓ Final shape: (5, 28)
✓ Fiscal Year range: FY2020 to FY2024
✓ Period coverage:
   FY2020: 2020-04-01 to 2020-06-30
   FY2021: 2020-07-01 to 2021-06-30
   FY2022: 2021-07-01 to 2022-06-30
   FY2023: 2022-07-01 to 2023-06-30
   FY2024: 2023-07-01 to 2024-06-30

Cleaned column names:
['maryland', 'allegany_county', 'anne_arundel_county', 'baltimore_city', 'baltimore_county', 'calvert_county', 'caroline_county', 'carroll_county', 'cecil_county', 'charles_county', 'dorchester_county', 'frederick_county', 'garrett_county', 'harford_county', 'howard_county', 'kent_county', 'montgomery_county', 'prince_george_s_county', 'queen_anne_s_county', 'somerset_county', 'st_mary_s_county', 'talbot_county', 'washington_county', 'wicomico_county', 'worcester_county', 'year', 'period_start', 'period_end']

Sample data:


,maryland,allegany_county,anne_arundel_county,baltimore_city,baltimore_county,calvert_county,caroline_county,carroll_county,cecil_county,charles_county,...,queen_anne_s_county,somerset_county,st_mary_s_county,talbot_county,washington_county,wicomico_county,worcester_county,year,period_start,period_end
0,-4982,-58,141,-2161,-928,106,-22,79,81,19,...,127,-28,133,36,-27,-89,135,2020,2020-04-01,2020-06-30
1,-8725,209,784,-6001,-2605,1200,141,1362,350,1662,...,879,92,229,501,693,507,1264,2021,2020-07-01,2021-06-30
2,1854,103,765,-5830,-2021,332,89,1527,1062,1280,...,861,218,-57,382,858,741,856,2022,2021-07-01,2022-06-30


## Step 3: Load Data to SQL Server Database

In [27]:
# Connect to SQL Server and load data
conn = mssql_python.connect(SQL_CONNECTION_STRING)
cursor = conn.cursor()

print("✓ Connecting to database...")

# Define column order: fiscal year, period dates, then all counties
# Get county columns (all numeric migration data)
county_cols = [col for col in df.columns if col not in ['year', 'period_start', 'period_end']]

# Create dynamic column definition based on actual data
column_definitions = []
column_definitions.append("FiscalYear INT")
column_definitions.append("PeriodStart DATE")
column_definitions.append("PeriodEnd DATE")

# Add columns for each county
for col in county_cols:
    # Remove '_county' suffix if present, then convert to PascalCase for SQL table
    clean_col = col.replace('_county', '')
    sql_col_name = ''.join(word.capitalize() for word in clean_col.split('_'))
    column_definitions.append(f"{sql_col_name} INT")  # Migration counts are integers

columns_sql = ',\n        '.join(column_definitions)

# Drop and recreate table to ensure correct schema
drop_table_sql = f"DROP TABLE IF EXISTS {TABLE_NAME}"
cursor.execute(drop_table_sql)
conn.commit()

create_table_sql = f"""
CREATE TABLE {TABLE_NAME} (
    {columns_sql}
)
"""

cursor.execute(create_table_sql)
conn.commit()
print("✓ Table created with updated schema - ready for data load")

# Prepare insert statement with all columns
# Map DataFrame columns to PascalCase SQL column names
column_mapping = {
    'year': 'FiscalYear',
    'period_start': 'PeriodStart',
    'period_end': 'PeriodEnd'
}

# Add county mappings
for col in county_cols:
    # Remove '_county' suffix if present, then convert to PascalCase
    clean_col = col.replace('_county', '')
    column_mapping[col] = ''.join(word.capitalize() for word in clean_col.split('_'))

# Define column order: fiscal year, period dates, then counties
ordered_cols = ['year', 'period_start', 'period_end'] + county_cols
sql_column_names = [column_mapping[col] for col in ordered_cols]
placeholders = ', '.join(['?'] * len(ordered_cols))
column_names = ', '.join(f'[{col}]' for col in sql_column_names)
insert_sql = f"INSERT INTO {TABLE_NAME} ({column_names}) VALUES ({placeholders})"

# Convert DataFrame to list of Python native type records
records = []
for _, row in df.iterrows():
    record = []
    for col in ordered_cols:
        val = row[col]

        # Normalize to native Python types for SQL driver
        # 1) Missing values
        if pd.isna(val):
            record.append(None)
            continue

        # 2) Explicitly handle NumPy / pandas scalar types first
        try:
            import numpy as np
        except ModuleNotFoundError:
            np = None

        if np is not None and isinstance(val, (np.generic, np.number)):
            record.append(val.item())
            continue

        # 3) Native Python primitives
        if isinstance(val, (int, float, str)):
            record.append(val)
            continue

        # 4) Other objects exposing .item()
        if hasattr(val, "item"):
            record.append(val.item())
            continue

        # 5) Fallback: stringify
        record.append(str(val))

    records.append(tuple(record))

# Insert records one by one
total_records = len(records)
total_inserted = 0

print(f"\n✓ Starting insert of {total_records:,} records...\n")

# Process records individually
for idx, record in enumerate(records, 1):
    cursor.execute(insert_sql, record)
    total_inserted += 1
    
    # Commit every 100 records and show progress
    if idx % 100 == 0 or idx == total_records:
        conn.commit()
        print(f"  ✓ Progress: {total_inserted:,} / {total_records:,} records inserted ({total_inserted/total_records*100:.1f}%)")

print(f"\n✓ All {total_inserted:,} records committed successfully")

cursor.close()
conn.close()

print(f"\n✓ Successfully loaded {total_inserted:,} records to {TABLE_NAME}")
print(f"✓ Fiscal Year range: FY{df['year'].min()} to FY{df['year'].max()}")

✓ Connecting to database...
✓ Table created with updated schema - ready for data load

✓ Starting insert of 5 records...

  ✓ Progress: 5 / 5 records inserted (100.0%)

✓ All 5 records committed successfully

✓ Successfully loaded 5 records to [Maryland].[dbo].[Migration]
✓ Fiscal Year range: FY2020 to FY2024


## Step 4: Verify Data in SQL Server

In [29]:
# Read data back from SQL to verify
query = f"SELECT * FROM {TABLE_NAME} ORDER BY FiscalYear"

# Suppress the pandas SQLAlchemy warning for mssql_python connections
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, message=".*pandas only supports SQLAlchemy.*")
    conn = mssql_python.connect(SQL_CONNECTION_STRING)
    try:
        df_verify = pd.read_sql(query, conn)
    finally:
        conn.close()

print(f"✓ Verification complete")
print(f"✓ Records in database: {len(df_verify):,}")
print(f"✓ Columns: {len(df_verify.columns)}")
print(f"\nFirst 5 records from database:")
df_verify.head()

✓ Verification complete
✓ Records in database: 5
✓ Columns: 28

First 5 records from database:


,FiscalYear,PeriodStart,PeriodEnd,Maryland,Allegany,AnneArundel,BaltimoreCity,Baltimore,Calvert,Caroline,...,Kent,Montgomery,PrinceGeorgeS,QueenAnneS,Somerset,StMaryS,Talbot,Washington,Wicomico,Worcester
0,2020,2020-04-01,2020-06-30,-4982,-58,141,-2161,-928,106,-22,...,-13,-1732,-2058,127,-28,133,36,-27,-89,135
1,2021,2020-07-01,2021-06-30,-8725,209,784,-6001,-2605,1200,141,...,311,-8998,-11763,879,92,229,501,693,507,1264
2,2022,2021-07-01,2022-06-30,1854,103,765,-5830,-2021,332,89,...,250,-341,-6755,861,218,-57,382,858,741,856
3,2023,2022-07-01,2023-06-30,11010,500,-833,-3681,-842,58,198,...,176,3158,-776,850,406,74,319,1012,294,516
4,2024,2023-07-01,2024-06-30,34591,252,416,237,3376,-34,293,...,188,8401,7059,1003,264,630,418,1438,481,442


In [30]:
# Quick preview of the data structure
print("Data preview:")
print(df_verify[['FiscalYear', 'PeriodStart', 'PeriodEnd', 'Maryland', 'Montgomery', 'BaltimoreCity']].to_string())

Data preview:
   FiscalYear PeriodStart   PeriodEnd  Maryland  Montgomery  BaltimoreCity
0        2020  2020-04-01  2020-06-30     -4982       -1732          -2161
1        2021  2020-07-01  2021-06-30     -8725       -8998          -6001
2        2022  2021-07-01  2022-06-30      1854        -341          -5830
3        2023  2022-07-01  2023-06-30     11010        3158          -3681
4        2024  2023-07-01  2024-06-30     34591        8401            237


## Summary

**Data Pipeline Complete! ✓**

This notebook successfully:
1. **Downloaded** Maryland Total Migration data from the Maryland Open Data Portal
2. **Cleaned and transformed** the data (normalized column names, converted data types, removed metadata)
3. **Loaded** the data into SQL Server database table `[Maryland].[dbo].[Migration]`
4. **Verified** the data was successfully stored

**Dataset Details:**
- **Source:** Maryland Open Data Portal, Maryland Department of Planning
- **Time Period:** Fiscal Years 2020-2024 (Maryland FY: July 1 - June 30)
- **Geographic Coverage:** Maryland statewide + all 23 counties + Baltimore City (24 jurisdictions)
- **Data Fields:** Fiscal Year, Period Start/End dates, net migration for each jurisdiction
- **Data Type:** Net migration (positive values = in-migration, negative values = out-migration)

**Fiscal Year Notes:**
- FY2020 = July 1, 2019 to June 30, 2020
- FY2021 = July 1, 2020 to June 30, 2021
- And so on...
- One partial period exists (April 1, 2020 to June 30, 2020) possibly due to COVID-19 data collection

**Next Steps:**
- Use this data for migration pattern analysis across Maryland jurisdictions
- Identify counties with net in-migration vs. out-migration
- Join with other Maryland datasets (e.g., Operating Budget, Per Capita Personal Income) for comprehensive analysis
- Create visualizations showing migration trends by fiscal year
- Analyze relationships between migration patterns, economic indicators, and housing costs